# Cvičení 
Cílem dnešního cvičení bude vyzkoušet si RAG.

Snowflake nabízí širokou škálu LLM modelů. My budeme pracovat s modelem _llama3.1-8b_.

## Část 1
Model zvolený pro tuto část je naučen na datech do prosince 2023. Naším cílem nyní bude oklasifikovat informace k některým událostem z posledních dvou let, abychom model naučili nové informace.

Před startem tohoto notebooku si ještě doinstalujte balíček _snowflake-ml-python_ - to provedete přes tlačítko _Packages_ napravo v horní liště stránky.

In [ ]:
import streamlit as st
import json
import pandas as pd

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

from snowflake.cortex import complete, CompleteOptions

session = get_active_session()

In [ ]:
# Pomocne fce

# Pomocna funkce na prevedeni radku do JSON formatu
def jsonify_row(row):
    return json.dumps(row.as_dict(), indent=2, ensure_ascii=False)


# Ocisteni uzivatelskeho jmena
def user_suffix(session):
    if name := session.get_current_user():
        return name[1:-1]
    # nemelo by nastat, ale at mame string
    import random
    return f"NONE_{random.randint(1,1000)}"

In [ ]:
categories = (
    session
    .table("NI_MLP_LAB.LAB.EVENT_CATEGORIES")
    .select(
        col("CATEGORY_ID").alias("ID"),
        col("CATEGORY_NAME").alias("CATEGORY")
    )
)

events_24 = (
    session
    .table("NI_MLP_LAB.LAB.EVENTS")
)

In [ ]:
# Zobrazeni kategorii
st.dataframe(categories, hide_index=True)

In [ ]:
# Zobrazeni letosnich udalosti
st.dataframe(events_24.order_by(col("EVENT_ID")))

Připrav systémový a uživatelský prompt, který použijeme ke klasifikaci jednotlivých událostí do jedné ze čtyř kategorií z tabulky _CATEGORIES_2024_. 

Zadefinuj také, aby byla odpověď vrácena v tomto JSON formátu:
```json
{
    "data": [
        {
            "id": "Hodnota EVENT_ID pro danou událost",
            "category": "Název kategorie (přesně jako je v CATEGORIES_2024)"
        }
        ...
    ]
}
```

In [ ]:
system_prompt = """You are a newspaper's editor assistant. Your task is to categorize texts provided by user into one of these four categories (each category is listed on separate line):
Sports
Culture
Science
Politics

User input will consist of one or more events, each in following JSON format:
{
    "event_id": integer (identifier),
    "event_text": string - text of event to be classified into one of categories
}

Return output in JSON format as a list (field "data"), result for each event is an object with fields event_id and category:
{
    "data": [
        {
            "event_id": "EVENT_ID of categorized text",
            "category": "Event category per class"
        }
    ]
}

Example output:
{
    "data": [
        {
            "event_id": 321,
            "category": "Culture"
        },
        {
            "event_id": 666,
            "category": "Sports"
        }
    ]
}
"""

In [ ]:
jsonify_row(events_24.first())

In [ ]:
# Zapsani vsech udalosti ve formatu JSON, ktere muzete pouzit v uzivatelskem promptu
# Samozrejme si muzete napsat i svoji vlastni serializaci
all_events = "\n".join(jsonify_row(row) for row in events_24.collect())

user_prompt = f"""
Classify please following events:
{all_events}
"""

In [ ]:
user_prompt

A nyní můžeme provolat LLM s našimi prompty. Zaexperimentujte si, jestli zvolený model bude správně kategorizovat a dodržovat požadovanou strukturu odpovědi při různých nastaveních hodnoty _temperature_ (musí být v intervalu <0,1>).

Objekt _CompleteOptions_ akceptuje kromě parametru _temperature_ také parametry:
- max_tokens
- top_p

Pro další podrobnosti k volání LLM modelů ve Snowparku si můžete prohlédnout dokumentaci:

[snowflake.cortex.complete](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/cortex/snowflake.cortex.complete)\
[snowflake.cortex.CompleteOptions](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/cortex/snowflake.cortex.CompleteOptions#snowflake.cortex.CompleteOptions)



In [ ]:
system_prompt

In [ ]:
model = "llama3.1-8b"
options = CompleteOptions(temperature=0.0)

classification = complete(
    model,
    [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    options=options
)

In [ ]:
classification

V dalších buňkách zpracujeme odpověď z LLM modelu. Předpokládáme, že formát odpovídá požadované struktuře - pokud ne, pravděpodobně to na něčem spadne :)

In [ ]:
# TODO: Trochu hloupa loads a dumps gymnastika
class_json_with_schema = json.loads(classification)
class_json_with_schema["schema"] = {
    "fields": [
        {
            "name": "event_id",
            "type": "integer"
        },
        {
            "name": "category",
            "type": "string"
        },
    ],
    "primaryKey": ["event_id"]
}

classification_pd = pd.json_normalize(
    json.loads(classification)["data"], 
    max_level=0
)

In [ ]:
# Workaround na bug s pridavanim uvozovek do nazvu sloupcu, kvuli kteremu pak 
# nefunguje dobre select sloupcu po joinu
user_tables_suffix = user_suffix(session)
classification_tmp_table = f"CLASS_TMP_{user_tables_suffix}"

session.write_pandas(
    classification_pd, 
    classification_tmp_table,
    database="NI_MLP_LAB",
    schema=f"LAB",
    quote_identifiers=False,
    auto_create_table=True,
    overwrite=True,
    table_type="temporary"
)

In [ ]:
classification_sp = session.read.table(classification_tmp_table)

output_table = (
    classification_sp.alias("csp")
    .join(
        events_24.alias("e24"),
        ["event_id"]
    )
    .join(
        categories.alias("cat"),
        ["category"]
    )
    .select(
        col("cat", "id").alias("CATEGORY_ID"),
        col("CATEGORY"),
        col("EVENT_ID"),
        col("EVENT_TEXT")
    )
)

In [ ]:
# Zobrazime si nase vysledky
# Pokud dostaneme prazdny vysledek, pravdepdobne bude problem v joinu na kategorizaci.
# Pouziva totiz join na presny nazev kategorie, takze mu muzou vadit napr. i bile znaky navic
# V tomto pripade si zkuste spustit predchozi bunku bez joinu na kategorie, jestli vam to projde,
# pokud ano, tak zkuste zacistit jmena kategorii z klasifikatoru nebu upravte prompty.

st.dataframe(output_table.orderBy("CATEGORY_ID", "EVENT_ID"))

Pokud jsme s výsledkem kategorizace spokojeni, uložíme si ho do pomocné tabulky a ještě ho otestujeme vůči kontrolní funkci. Pokud nám test projde, můžeme pokračovat do dalšího notebooku.

In [ ]:
# pokud chcete prepsat existujici tabulku, nastavte mode="overwrite"
categorization_table = f"NI_MLP_LAB.MLP_{user_tables_suffix}.EVENT_CATEGORIZATION_{user_tables_suffix}"

output_table.write.save_as_table(
    categorization_table,
    mode="overwrite",
)

In [ ]:
-- Zavolame kontrolni proceduru
CALL NI_MLP_LAB.LAB.control_classification(
    SYSTEM$REFERENCE('TABLE', '{{categorization_table}}', 'SESSION', 'SELECT')
);

In [ ]:
# If needed, we may get result of previous SQL cell to Python like this
control_df = cell5.to_df()
control_df.where(col("IS_CORRECT") == False)